# Reactor Kinetics Example 

Jialu Wang (jwang44@nd.edu) and Alex Dowling (adowling@nd.edu)

University of Notre Dame

This notebook conducts design of experiments for a reactor kinetics experiment with the Pyomo.DOE.
    

In [1]:
import matplotlib.pyplot as plt
from pyomo.environ import *
from pyomo.dae import *

import numpy as np
from scipy.interpolate import interp2d
import pandas as pd 
from itertools import permutations, product, combinations
import idaes

from fim_doe import *
#from pyomo.contrib.sensitivity_toolbox.sens import sipopt
#from idaes.apps.uncertainty_propagation.sens import get_dsdp
#from pyomo.contrib.sensitivity_toolbox.sens import get_dsdp

## Define Reaction Example Mathematical Model

Consider two chemical reactions that converts molecule $A$ to desired product $B$ and a less valuable side-product $C$.

$A \overset{k_1}{\rightarrow} B \overset{k_2}{\rightarrow} C$

Our ultimate goals is to design a large-scale continous reactor that maximizes the production of $B$. This general sequential reactions problem is widely applicable to CO$_2$ capture and industry more broadly (petrochemicals, pharmasuticals, etc.).

The rate laws for these two chemical reactions are:

$r_A = -k_1 C_A$

$r_B = k_1 C_A - k_2 C_B$

$r_C = k_2 C_B$

Here, $C_A$, $C_B$, and $C_C$ are the concentrations of each species. The rate constants $k_1$ and $k_2$ depend on temperature as follows:

$k_1 = A_1 \exp{\frac{-E_1}{R T}}$

$k_2 = A_2 \exp{\frac{-E_2}{R T}}$

$A_1, A_2, E_1$, and $E_2$ are fitted model parameters. $R$ is the ideal-gas constant and $T$ is absolute temperature.

Using the **CCSI$^2$ toolset**, we would like do the following perform:

Perform **uncertainty quantification** and **design of experiments** on a small-scale **batch reactor** to infer parameters $A_1$, $A_2$, $E_1$, and $E_2$.

### Batch reactor

The concenrations in a batch reactor evolve with time per the following differential equations:

$$ \frac{d C_A}{dt} = r_A = -k_1 C_A $$

$$ \frac{d C_B}{dt} = r_B = k_1 C_A - k_2 C_B $$

$$ \frac{d C_C}{dt} = r_C = k_2 C_B $$

This is a linear system of differential equations. Assuming the feed is only species $A$, i.e., 

$$C_A(t=0) = C_{A0} \quad C_B(t=0) = 0 \quad C_C(t=0) = 0$$

When the temperature is constant, it leads to the following analytic solution:

$$C_A(t) = C_{A,0} \exp(-k_1 t)$$

$$C_B(t) = \frac{k_1}{k_2 - k_1} C_{A,0} \left[\exp(-k_1 t) - \exp(-k_2 t) \right]$$

$$C_C(t) = C_{A,0} - \frac{k_2}{k_2 - k_1} C_{A,0} \exp(-k_1 t) + \frac{k_1}{k_2 - k_1} \exp(-k_2 t) C_{A,0} = C_{A,0} - C_{A}(t) - C_{B}(t)$$

In [2]:
from reactor_kinetics import *

Three versions of this model is accomplished: 

Dynamic-DAE model: Temperature varying model discretized and integrated by Pyomo.DAE

Constant-DAE model: Temperature constant model discretized and integrated by Pyomo.DAE

Constant-analytical model: Temperature constant model discretized manually and using the analytical expressions for state variables.

In [3]:

createmod = create_model
#args_ = [True, False, False]
disc = disc_for_measure
t_control = [0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1]
    
# design variable and its control time set
dv_pass = {'CA0': [0],'T': t_control}
    

# Define measurement time points
t_measure = [0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1]
#t_measure_ca = [0, 0.25, 0.5, 0.75, 1]
#t_measure_cb = [0, 0.125, 0.375, 0.625, 0.875, 1]
#measure_pass = {'CA': t_measure, 'CB': t_measure, 'CC': t_measure}
#measure_pass = {'C':{"'CA'": t_measure, "'CB'": t_measure, "'CC'": t_measure}}
measure_pass = {'C':{'CA': t_measure, 'CB': t_measure, 'CC': t_measure}}
#measure_pass = {'C':{'CA': t_measure_ca, 'CB': t_measure_cb, 'CC': t_measure}}
measure_class =  Measurements(measure_pass)
'''
comb = [t_measure, t_measure_ca, t_measure_cb]
#unique = list(set(t_measure_ca+t_measure_cb))
d = []
for i in comb:
    d += i
    unique = list(set((d)))

print(unique)
'''

All measurements are flattened.
Flatten measurement name: ['C_index_CA', 'C_index_CB', 'C_index_CC']
Flatten measurement variance: {'C_index_CA': 1, 'C_index_CB': 1, 'C_index_CC': 1}


'\ncomb = [t_measure, t_measure_ca, t_measure_cb]\n#unique = list(set(t_measure_ca+t_measure_cb))\nd = []\nfor i in comb:\n    d += i\n    unique = list(set((d)))\n\nprint(unique)\n'

In [4]:
# Define parameter nominal value 
parameter_dict = {'A1': 84.79085853498033, 'A2': 371.71773413976416, 'E1': 7.777032028026428, 'E2': 15.047135137500822}

def generate_exp(t_set, CA0, T):  
    '''Generate experiments. 
    t_set: time control set for T.
    CA0: CA0 value
    T: A list of T 
    '''
    assert(len(t_set)==len(T)), 'T should have the same length as t_set'
    
    T_con_initial = {}
    for t, tim in enumerate(t_set):
        T_con_initial[tim] = T[t]
        
    dv_dict_overall = {'CA0': {0: CA0},'T': T_con_initial}
    return dv_dict_overall

In [5]:
# empty prior
prior_all = np.zeros((4,4))


# add prior information
#prior_5_300 = pd.read_csv('fim_5_300_scale.csv')
#prior_5_300_500 = pd.read_csv('fim_5_300_500_scale.csv')

#prior_all = prior_5_300


prior_pass=np.asarray(prior_all)

#L_initials = np.linalg.cholesky(prior_pass)
#print(L_initials)

print('The prior information FIM:', prior_pass)
print('Prior Det:', np.linalg.det(prior_pass))
print('Eigenvalue of the prior experiments FIM:', np.linalg.eigvals(prior_pass))
print('Eigenvalue of the prior experiments FIM:', np.linalg.eigh(prior_pass)[1])

The prior information FIM: [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
Prior Det: 0.0
Eigenvalue of the prior experiments FIM: [0. 0. 0. 0.]
Eigenvalue of the prior experiments FIM: [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


## Compute FIM 

This method computes an MBDoE optimization problem with no Degree of Freedom.

In [9]:
# choose from 'simultaneous', 'sequential', 'sipopt'
sensi_opt = 'sequential_finite'
#sensi_opt = 'sequential_sipopt'
#sensi_opt = 'sequential_kaug'
#sensi_opt = 'direct_kaug'

#if sensi_opt == 'direct_kaug':
#    args_[2] = True
#else:
#    args_[2] = False
    

# Define experiments
#if (model_opt=='dynamic-DAE') or (model_opt=='dynamic-DAE-measure'):
#exp1 = generate_exp(t_control, 5, [570.21, 300, 300, 300, 300, 300, 300, 300, 300])
exp1 = generate_exp(t_control, 5, [300, 300, 300, 300, 300, 300, 300, 300, 300])
#else: 
#    exp1 = generate_exp(t_control, 5, [300])

print('Design variable:', exp1)

Design variable: {'CA0': {0: 5}, 'T': {0: 300, 0.125: 300, 0.25: 300, 0.375: 300, 0.5: 300, 0.625: 300, 0.75: 300, 0.875: 300, 1: 300}}


In [10]:
doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 measure_class, createmod,
                                prior_FIM=prior_pass, discretize_model=disc, args=None)

if_s = True

result = doe_object.compute_FIM(exp1,mode=sensi_opt, FIM_store_name = 'dynamic.csv', 
                                store_output = 'store_output', read_output=None,
                                scale_nominal_param_value=if_s, formula='central')


result.calculate_FIM(doe_object.design_values)


Sensitivity information is scaled by its corresponding parameter nominal value.
This scenario: {'A1': {0: 84.8756493935153}, 'A2': {0: 371.71773413976416}, 'E1': {0: 7.777032028026428}, 'E2': {0: 15.047135137500822}, 'jac-index': {'A1': [0], 'A2': [0], 'E1': [0], 'E2': [0]}, 'eps-abs': {'A1': 0.0848756493935153, 'A2': 0.37171773413976417, 'E1': 0.007777032028026428, 'E2': 0.015047135137500823}, 'scena-name': [0]}
Ipopt 3.13.2: linear_solver=ma57
halt_on_ampl_error=yes
max_iter=3000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Fram

Output this time:  [5.0, 3.128100487643164, 1.9570025321507007, 1.2243401150153699, 0.7659717821490261, 0.4792073410357073, 0.2998017434178557, 0.18756199593856357, 0.11734255417375321, 0.0, 1.7635365243639423, 2.6806122145737454, 3.0877905607990144, 3.193555776848441, 3.1264824020080604, 2.965348014661868, 2.757951639147301, 2.532868394480739, 0.0, 0.10836298799289328, 0.3623852532755539, 0.6878693241856151, 1.0404724410025326, 1.3943102569562322, 1.734850241920277, 2.054486364914135, 2.3497890513455078]
This scenario: {'A1': {0: 84.79085853498033}, 'A2': {0: 371.71773413976416}, 'E1': {0: 7.784809060054453}, 'E2': {0: 15.047135137500822}, 'jac-index': {'A1': [0], 'A2': [0], 'E1': [0], 'E2': [0]}, 'eps-abs': {'A1': 0.08479085853498033, 'A2': 0.37171773413976417, 'E1': 0.007784809060054453, 'E2': 0.015047135137500823}, 'scena-name': [0]}
Ipopt 3.13.2: linear_solver=ma57
halt_on_ampl_error=yes
max_iter=3000


******************************************************************************

Output this time:  [5.0, 3.128100487643164, 1.9570025321507003, 1.22434011501537, 0.7659717821490263, 0.4792073410357074, 0.29980174341785576, 0.1875619959385635, 0.11734255417375307, 0.0, 1.7642672787445501, 2.6829574722130474, 3.0920515016041543, 3.199709278680381, 3.134336951298549, 2.97463632871135, 2.768384347170444, 2.5441639264016307, 0.0, 0.10763223361228579, 0.36003999563625244, 0.6836083833804756, 1.0343189391705927, 1.3864557076657429, 1.725561927870794, 2.0440536568909926, 2.338493519424616]
This scenario: {'A1': {0: 84.70606767644534}, 'A2': {0: 371.71773413976416}, 'E1': {0: 7.777032028026428}, 'E2': {0: 15.047135137500822}, 'jac-index': {'A1': [0], 'A2': [0], 'E1': [0], 'E2': [0]}, 'eps-abs': {'A1': 0.08470606767644534, 'A2': 0.37171773413976417, 'E1': 0.007777032028026428, 'E2': 0.015047135137500823}, 'scena-name': [0]}
Ipopt 3.13.2: linear_solver=ma57
halt_on_ampl_error=yes
max_iter=3000


******************************************************************************
T

Output this time:  [5.0, 3.1281004876431657, 1.9570025321507023, 1.224340115015371, 0.7659717821490261, 0.47920734103570745, 0.29980174341785576, 0.18756199593856354, 0.1173425541737531, 0.0, 1.7637448440728156, 2.681280653497789, 3.0890047456944947, 3.1953088745425484, 3.1287196092056884, 2.967992980410903, 2.760921771878387, 2.5360833789476414, 0.0, 0.10815466828401792, 0.3617168143515083, 0.6866551392901342, 1.0387193433084259, 1.3920730497586042, 1.7322052761712412, 2.0515162321830496, 2.3465740668786053]
This scenario: {'A1': {0: 84.79085853498033}, 'A2': {0: 371.71773413976416}, 'E1': {0: 7.769254995998401}, 'E2': {0: 15.047135137500822}, 'jac-index': {'A1': [0], 'A2': [0], 'E1': [0], 'E2': [0]}, 'eps-abs': {'A1': 0.08479085853498033, 'A2': 0.37171773413976417, 'E1': 0.0077692549959984016, 'E2': 0.015047135137500823}, 'scena-name': [0]}
Ipopt 3.13.2: linear_solver=ma57
halt_on_ampl_error=yes
max_iter=3000


*************************************************************************

Output this time:  [5.0, 3.1281004876431644, 1.957002532150701, 1.224340115015371, 0.7659717821490261, 0.4792073410357075, 0.2998017434178558, 0.1875619959385635, 0.1173425541737532, 0.0, 1.7630105811377832, 2.6789250910651563, 3.0847268984955987, 3.189133683126401, 3.120840972406899, 2.9586805600431965, 2.7504670265012545, 2.5247695573328652, 0.0, 0.10888893121905242, 0.36407237678414256, 0.6909329864890306, 1.044894534724573, 1.399951686557394, 1.7415176965389476, 2.0619709775601818, 2.357887888493382]
Build time with sequential_finite mode [s]: 0.03111720085144043
Solve time with sequential_finite mode [s]: 0.8737852573394775
Total wall clock time [s]: 1.4387235641479492
Splitted jacobian: {'A1': [0.0, -1.467116588349304, -1.835715448094244, -1.7226910323040465, -1.437000544313338, -1.1237708966554127, -0.8436647297968003, -0.6157828391143705, -0.4402815219688022, 0.0, 1.3746483253118669, 1.5721380344269063, 1.2968080739290677, 0.8889616524363486, 0.49889489113263963, 0.181624832987

In [11]:
print('======Result summary======')
print('Four design criteria log10() value:')
print('A-optimality:', np.log10(result.trace))
print('D-optimality:', np.log10(result.det))
print('E-optimality:', np.log10(result.min_eig))
print('Modified E-optimality:', np.log10(result.cond))

======Result summary======
Four design criteria log10() value:
A-optimality: 2.962954001638135
D-optimality: -15.72265453005471
E-optimality: -10.672887373949454
Modified E-optimality: 13.505276842965191
